#Greetings!

Hello, i wrote this notebook as exercise since I'm now interested in data science. This notebook is about Data Science Job Salaries. The dataset and learning source provided at the bottom of this notebook.

#1. Import and Read the .csv file

In [1]:
import numpy as np
import pandas as pd

import plotly.express as px
import matplotlib.pyplot as plt
import os
from wordcloud import WordCloud

In [2]:
pip install pycountry

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.1 MB 7.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=10681845 sha256=a89f9b7982d05c50c5e30dd6937a7d3d6ac0a32e96d2faf68be2543139d3e23f
  Stored in directory: /root/.cache/pip/wheels/0e/06/e8/7ee176e95ea9a8a8c3b3afcb1869f20adbd42413d4611c6eb4
Successfully built pycountry


In [3]:
import pycountry

In [4]:
read = pd.read_csv('ds_salaries.csv',
                   index_col=0)
df = read.copy()

#2. Looking at the properties of the dataset

Tipe of the datasets

In [13]:
df.dtypes

work_year              int64
experience_level      object
employment_type       object
job_title             object
salary                 int64
salary_currency       object
salary_in_usd          int64
employee_residence    object
remote_ratio           int64
company_location      object
company_size          object
dtype: object

Show the first five rows from the datasets

In [37]:
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L


From dataset sample below, the 'experience_level', 'employment_type'. 'remote_ratio' value are coded by the company. For outside reader, it's necessary to make it easier to understand. We will replace the values later.





We want to know from what year the data is

In [64]:
df.get(['work_year']).value_counts()

work_year
2022         318
2021         217
2020          72
dtype: int64

The data year ranging from 2020 to 2021, it's from the pandemic era

Next, Let's check whether there is a missing values in the datasets

In [20]:
df.isnull().sum()

work_year             0
experience_level      0
employment_type       0
job_title             0
salary                0
salary_currency       0
salary_in_usd         0
employee_residence    0
remote_ratio          0
company_location      0
company_size          0
dtype: int64

Count the unique value from each columns

In [21]:
df.nunique()

work_year               3
experience_level        4
employment_type         4
job_title              50
salary                272
salary_currency        17
salary_in_usd         369
employee_residence     57
remote_ratio            3
company_location       50
company_size            3
dtype: int64

#3. Preprocessing some data

First we need to know what code is used for 'experience_level', 'employment_type'. and 'remote_ratio' column.

In [40]:
df.get(['experience_level']).value_counts()

experience_level
SE                  280
MI                  213
EN                   88
EX                   26
dtype: int64

For the experience level we knows that
1.   EX = Experienced
2.   SE = Senior-Level
3.   MI = Mid-Level
4.   EN = Entry-Level



In [41]:
df.get(['employment_type']).value_counts()

employment_type
FT                 588
PT                  10
CT                   5
FL                   4
dtype: int64

For the employment type we knows that
1.   FT = Full-Time
2.   PT = Part-Time
3.   CT = Contract
4.   FL = Freelance



In [42]:
df.get(['remote_ratio']).value_counts()

remote_ratio
100             381
0               127
50               99
dtype: int64

The remote ratio describe whether the employe is working remote, hybrid or from office as follows
1.   0 = Work from office (WFO)
2.   50 = Hybrid
3.   100 = Remote


Replacing some parameter value to make it easier to understand

In [5]:
df.experience_level.replace(['EN', 'MI', 'SE', 'EX'], ['Entry', 'Mid', 'Senior', 'Executive'], inplace=True) #expand the acronyms for the experience level column
df.employment_type.replace(['FT', 'PT', 'CT', 'FL'], ['Full-Time', 'Part-Time', 'Contract', 'Freelance'], inplace=True)
df.remote_ratio.replace([0, 50, 100], ['WFO', 'Hybrid', 'Remote'], inplace = True) #replce number value on remote ratio to more easy to understand word

Since the 'employee_residence' column is coded in ALPHA-2, we will convert it into a country name so the reader doesn't need to look up on the table 

First convert the code into country name and add into new coloumn int the dataframe as 'employee_country'

In [17]:
alpha_2_code = [i.alpha_2 for i in list(pycountry.countries)]

def convert_code(df):
  return pycountry.countries.get(alpha_2=df['employee_residence']).name

df['employee_country'] = df.apply(convert_code, axis=1)

Let's check the updated dataset

In [86]:
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2020,Mid,Full-Time,Data Scientist,70000,EUR,79833,DE,WFO,DE,L
1,2020,Senior,Full-Time,Machine Learning Scientist,260000,USD,260000,JP,WFO,JP,S
2,2020,Senior,Full-Time,Big Data Engineer,85000,GBP,109024,GB,Hybrid,GB,M
3,2020,Mid,Full-Time,Product Data Analyst,20000,USD,20000,HN,WFO,HN,S
4,2020,Senior,Full-Time,Machine Learning Engineer,150000,USD,150000,US,Hybrid,US,L


# 4. Analysis

## 1. Univarient Analysis


> Univariate analysis is used for describe a single variable data. Since it's a single variable analysis, i doesn't explain the realtionship of the data. This analysis is used to summarize and find patterns in the data.


---
https://www.statisticshowto.com/univariate/





### 1. Bar chart of the employment types

In [55]:
px.histogram(df,
             x='employment_type',
             histnorm='percent',
             text_auto='.2f',
             template='plotly_dark',
             title='Percentage of the Employment Types')

From the histogram below we could see that most of the employees are working full-time.

### 2. Are they working from the office, hybrid or maybe they're working remotely?

In [60]:
px.histogram(df,
             x='remote_ratio',
             histnorm='percent',
             color_discrete_sequence=px.colors.qualitative.Pastel,
             template='plotly_dark',
             title='The Work Type'
             )

62% of the employee are working remotely. Since the data cames from the pandemic era, we want to know if the pandemic has affected the working type of the employee.

In [69]:
px.histogram(df,
             x='remote_ratio',
             color='work_year',
             histnorm='percent',
             barmode='group',
             color_discrete_sequence=px.colors.qualitative.Pastel,
             template='plotly_dark',
             title='The Work Type'
             )



1.   In 2020 half of the work remotely with the rest either working from office or working with hybrid shift
2.   Then 2021 number of employee working from office decreasing thus increasing the number of employee either working hybrid or remote.
3.   The pandemic slows down in 2022, the hybrid shift is no longer favourable, a quarter of the population working from office. Since working remotely becoming more easier, 71% of employee then choose to work remotely.



### 3. Where is these guys came from?

We want to know these data science people came from

In [38]:
px.histogram(df,
             x = df.employee_country.sort_values(),
             histnorm='percent',
             text_auto='.1f',
             color_discrete_sequence=px.colors.qualitative.Safe,
             template='plotly_dark',
             title='Count Employee residece in each country'
             )

The histogram show us that most of the employee came from US. The other came from UK, India, Canada, and Germany.

### 4. Why the US has many Data Scientist? Number of company maybe?

In [41]:
def convert_code(df):
  return pycountry.countries.get(alpha_2=df['company_location']).name

df['company_country'] = df.apply(convert_code, axis=1)

In [42]:
px.histogram(df,
             x=df.company_country.sort_values(),
             histnorm='percent',
             text_auto='.1f',
             color_discrete_sequence=px.colors.qualitative.Safe,
             template='plotly_dark',
             title='Company Location'
             )

For the US and UK, since they have a lot of company they also have a lot of number of data scientist. Amazingly, India has less company compared to Canada and Germany. Interesting...

### 5. We want to know in which data field is these employee

In [44]:
px.histogram(df,
             x=df.job_title.sort_values(),
             color='experience_level',
             histnorm='percent',
             height=900,
             barmode='group',
             color_discrete_sequence=px.colors.qualitative.Pastel,
             template='plotly_dark',
             text_auto=True,
             title='The Data Field and Employee Experience'
             )

From the histogram we have:


1.   Top three data field are Data Scientist, Data Engineer, and Data Analyst.
2.   Data scientist field have more Senior and Executive level employee. Data Engineer and Data Analyst are stacked with new employee by looking at the number of Entry and Mid level employee.
3.   There are only ten field that have employee with executive experience.



## 2. Multivarient Analysis

Statistical analysis that analyze the relationship between multivariate measurement parameter. It used to analyze the effect betwwen multuiple populations. This analysis could be used to test the hypothesis between variables.


---
https://www.sciencedirect.com/topics/medicine-and-dentistry/multivariate-analysis#:~:text=Multivariate%20analysis%20is%20conceptualized%20by,important%20to%20the%20experiment's%20understanding.


### 1. Scatter Plot of the company location and employee residence

In [48]:
px.scatter(
    df,
    x=df.employee_country.sort_values(),
    y=df.company_country.sort_values(),
    color='remote_ratio',
    height=1000,
    labels={"x": 'Employee Residence',
            "y": 'Companay Location'},
    color_discrete_sequence=px.colors.qualitative.Pastel,
    template='plotly_dark',
    title='Relationship between Employee Residence and Company Location with type of work (WFO, Hybrid, or Remote'
)

Employee from outside of their company country likely to work remotely. But US employee also love to work remotely, maybe their company is in another state that they think the company is in another country... 

### 2. Experience and Money, are they related?

In [50]:
px.scatter(
    df,
    x='salary_in_usd',
    y='experience_level',
    size='salary_in_usd',
    hover_name='job_title',
    color='job_title',
    color_discrete_sequence=px.colors.qualitative.Pastel_r,
    template='plotly_dark',
    animation_frame='work_year',
    title='Experience and Money'
).update_yaxes(categoryarray=['Entry', 'Mid', 'Senior', 'Executive'])

From the scatter plot we could observe that


1.   Even with higher experience, the salary could be the same level with employee with greener experience.
2.   Data Science field is working field with the most high salary.
3.   In 2022, the average salary range is 100k - 200k USD.
2.   From 2020 to 2022, the number of employee in data field is increasing.



# 5. Conclusion

The data field is high paid job. The average salary range is 100k - 200k USD. This kind of job popularity increase as the number of Entry and Mid level employee increasing from 2020 and 2022. Since most of the employee are from different country, working remotely is popular among employee working in the data field.

# References



1.   [Datasets](https://www.kaggle.com/code/priyark/data-science-job-salaries-plotly-interactive-graph/data)
2.   [Notebook](https://www.kaggle.com/code/priyark/data-science-job-salaries-plotly-interactive-graph)
3. [Documentation](https://pandas.pydata.org/docs/reference/frame.html)
4. [Converting country code](https://stackoverflow.com/questions/53923433/how-to-get-country-name-from-country-abbreviation-in-python-with-mix-of-alpha-2)

